In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson
from string import ascii_uppercase as alphabet
import numpy as np

In [2]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_euro_matches.csv')
df_fixture = pd.read_csv('clean_fifa_euro_fixture.csv')

In [3]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()

# New model with distance parameter

In [4]:
def predict_points2(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        
        # mean of all distance since 1968
        if abs(points_home - points_away) < 0.8:
            return (1, 1)  # Match nul
        elif points_home > points_away:
            return (3, 0)  # Victoire pour la maison
        else:
            return (0, 3)  # Victoire pour l'extérieur
    else:
        return (0, 0)

In [5]:
df_fixture_group_36 = df_fixture[:36].copy()
df_fixture_knockout = df_fixture[36:44].copy()
df_fixture_quarter = df_fixture[44:48].copy()
df_fixture_semi = df_fixture[48:50].copy()
df_fixture_final = df_fixture[50:].copy()

In [6]:
for group in dict_table:
    teams_in_group = dict_table[group]['Équipe'].values
    df_fixture_group_6 = df_fixture_group_36[df_fixture_group_36['home'].isin(teams_in_group)] # Represent 6 matches in 1 group
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points2(home, away)  # Adjusted predict_points function
        dict_table[group].loc[dict_table[group]['Équipe'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Équipe'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Équipe', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [7]:
dict_table['Group A']

,Équipe,Pts
0,Allemagne,9
1,Écosse,2
2,Hongrie,2
3,Suisse,2


In [8]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Équipe']
    runners_up = dict_table[group].loc[1, 'Équipe']
    df_fixture_knockout.replace({f'1{group}':group_winner, f'2{group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'

In [9]:
# Replace winner and 2nd place
def replace_teams(row):
    home = row['home']
    away = row['away']

    # Extraction group letter thanks to 'home' and 'away'
    group_home = home[1:]
    group_away = away[1:]
    

    #Replace name team according to the group
    if home.startswith('1'):
        home = dict_table[f'Group {group_home}'].loc[0, 'Équipe']
    if away.startswith('1'):
        away = dict_table[f'Group {group_away}'].loc[0, 'Équipe']
    if home.startswith('2'):
        home = dict_table[f'Group {group_home}'].loc[1, 'Équipe']
    if away.startswith('2'):
        away = dict_table[f'Group {group_away}'].loc[1, 'Équipe']

    # Retourner les noms des équipes modifiés
    return home, away

# Appliquer la fonction de remplacement sur les lignes du DataFrame
df_fixture_knockout[['home', 'away']] = df_fixture_knockout[['home', 'away']].apply(replace_teams, axis=1, result_type='expand')
df_fixture_knockout['away'].replace('3A/D/E/F', 'Ukraine', inplace=True)
df_fixture_knockout['away'].replace('3D/E/F', 'Pologne', inplace=True)
df_fixture_knockout['away'].replace('3A/B/C', 'Hongrie', inplace=True)
df_fixture_knockout['away'].replace('3A/B/C/D', 'Croatie', inplace=True)

In [10]:
dict_table[('Group F')]

,Équipe,Pts
0,Portugal,6
1,Tchéquie,3
2,Turquie,0
3,Géorgie,0


In [11]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points2(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [12]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
36,Allemagne,Match 37,Danemark,2024,Allemagne
37,Écosse,Match 38,Croatie,2024,Croatie
38,Espagne,Match 39,Ukraine,2024,Espagne
39,Angleterre,Match 40,Pologne,2024,Angleterre
40,Portugal,Match 41,Hongrie,2024,Portugal
41,France,Match 42,Slovaquie,2024,France
42,Belgique,Match 43,Croatie,2024,Croatie
43,Pays-Bas,Match 44,Tchéquie,2024,Pays-Bas


In [13]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = int(df_fixture_round_1.loc[index, 'score'].split()[1])
        df_fixture_round_2.replace({f'W{match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [14]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
44,Espagne,Match 45,Allemagne,2024,?
45,Portugal,Match 46,France,2024,?
46,Croatie,Match 47,Pays-Bas,2024,?
47,Angleterre,Match 48,Croatie,2024,?


In [15]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
44,Espagne,Match 45,Allemagne,2024,Allemagne
45,Portugal,Match 46,France,2024,France
46,Croatie,Match 47,Pays-Bas,2024,Pays-Bas
47,Angleterre,Match 48,Croatie,2024,Croatie


In [16]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
48,Allemagne,Match 49,France,2024,?
49,Pays-Bas,Match 50,Croatie,2024,?


In [17]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
48,Allemagne,Match 49,France,2024,France
49,Pays-Bas,Match 50,Croatie,2024,Pays-Bas


In [18]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
50,France,Match 51,Pays-Bas,2024,?


In [19]:
get_winner(df_fixture_final)

,home,score,away,year,winner
50,France,Match 51,Pays-Bas,2024,Pays-Bas
